# Biological Classification problem with phenotype

Note: I don't own the data. It is gitignored to protect patent

In [ ]:
import pandas as pd

df = pd.read_csv("plant_traits_espece_ee.csv")
df.columns = ['organs', 'trait', 'trait_code', 'finer_code/detail'] + df.columns.to_list()[4:]
df = df.transpose()
df = df.reset_index()

n = len(df.iloc[0])

import numpy as np

# p_organs, p_trait, p_trait_code = [np.nan] * 3


for j in range(5, n):
    if pd.isnull(df.iloc[0, j]):
        df.iloc[0, j] = p_organs
    else:
        p_organs = df.iloc[0, j]
    
    if pd.isnull(df.iloc[1, j]):
        df.iloc[1, j] = p_trait
    else:
        p_trait = df.iloc[1, j]

    if pd.isnull(df.iloc[2, j]):
        df.iloc[2, j] = p_trait_code
    else:
        p_trait_code = df.iloc[2, j]


    df.iloc[3, j] = ">".join(map(str, (p_organs, p_trait, p_trait_code)))

df.iloc[0:3,4] = np.nan


df2 = df.iloc[3:,:]
# df2[0] = df2[0].astype(str)
df2.iloc[0,0] = 'organs>trait>trait_code>detail'
new_header = df2.iloc[0]
df2 = df2[1:] 
df2.columns = new_header
# df2

In [ ]:
df2

we can do some direct filtering

In [ ]:
df2.iloc[~pd.isnull(df2['usage>usage.material>staminal.tube'].values)]

Prediction(without reweighting)
the family don't share that much info, and the result is poor due to insufficient data.

In [ ]:
df_w_label = df2.dropna(subset='Family')
df_w_label.iloc[:,5:] = [[df_w_label.iloc[i,j] if pd.isnull(df_w_label.iloc[i,j]) else 1 for j in range(5,n)] for i in range(len(df_w_label))]
X = df_w_label.iloc[:,5:]
y = df_w_label['Family']

# from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier
m = RandomForestClassifier(n_jobs=4, random_state=3)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss


X_train,  X_test,y_train, y_test = train_test_split(X, y, test_size=.5, random_state=3)# stratify=y)
# print(X_train)
# print(y_train)
m.fit(X_train, y_train)
y_pred = m.predict(X_test)
y_prob = m.predict_proba(X_test)
y_prob ={m.classes_[i]:[y_prob[j][i]for j in range(len(y_prob))] for i in range(len(m.classes_))} 


perf = accuracy_score(y_test, y_pred)
conf = confusion_matrix(y_test, y_pred)
# log_loss(y_test, y_prob)
perf,# conf



In [ ]:
y_prob

In [ ]:
m.classes_

This is not valid because the classifier is ignoring all missings and there is no 2 values for non nas. Here we think na= false, redo it

In [ ]:
df_w_label = df2.dropna(subset='Family')
df_w_label.iloc[:,5:] = [[0 if pd.isnull(df_w_label.iloc[i,j]) else 1 for j in range(5,n)] for i in range(len(df_w_label))]
X = df_w_label.iloc[:,5:]
y = df_w_label['Family']

# from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
# from sklearn.tree import DecisionTreeClassifier
m = RandomForestClassifier(n_jobs=4, random_state=3)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss


X_train,  X_test,y_train, y_test = train_test_split(X, y, test_size=.5, random_state=3)# stratify=y)
# print(X_train)
# print(y_train)
m.fit(X_train, y_train)
y_pred = m.predict(X_test)
y_prob = m.predict_proba(X_test)
y_prob ={m.classes_[i]:[y_prob[j][i]for j in range(len(y_prob))] for i in range(len(m.classes_))} 


perf = accuracy_score(y_test, y_pred)
conf = confusion_matrix(y_test, y_pred)
# log_loss(y_test, y_prob)
perf,# conf



here 50% probability we get correct family, is this bad? somewhat, but remember we get more than 2 classes.
If we have distinct rows, we can try to over fit this--which is equivalent to coding a tree.
now we want to formulate this as stepwise question. this needs to access all the nodes.




In [ ]:
df_w_label = df2.dropna(subset='Family')
df_w_label.iloc[:,5:] = [[0 if pd.isnull(df_w_label.iloc[i,j]) else 1 for j in range(5,n)] for i in range(len(df_w_label))]
X = df_w_label.iloc[:,5:]
y = df_w_label['Family']

# from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
m = DecisionTreeClassifier()

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss


X_train,  X_test,y_train, y_test = train_test_split(X, y, test_size=.5, random_state=3)# stratify=y)
# print(X_train)
# print(y_train)
m.fit(X_train, y_train)
y_pred = m.predict(X_test)
y_prob = m.predict_proba(X_test)
y_prob ={m.classes_[i]:[y_prob[j][i]for j in range(len(y_prob))] for i in range(len(m.classes_))} 


perf = accuracy_score(y_test, y_pred)
conf = confusion_matrix(y_test, y_pred)
# log_loss(y_test, y_prob)
perf,# conf



instead of rewriting the entire structure, this is equivalent of overfitting a decision tree in sklearn(do not worry, it can stop prematurely)

In [ ]:
m = DecisionTreeClassifier(max_depth=1000)
m.fit(X,y)

the code from sklearn shows the structure, we can build an *interactive  program* with it

In [ ]:
clf = m




n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold
values = clf.tree_.value

node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
is_leaves = np.zeros(shape=n_nodes, dtype=bool)
stack = [(0, 0)]  # start with the root node id (0) and its depth (0)
while len(stack) > 0:
    # `pop` ensures each node is only visited once
    node_id, depth = stack.pop()
    node_depth[node_id] = depth

    # If the left and right child of a node is not the same we have a split
    # node
    is_split_node = children_left[node_id] != children_right[node_id]
    # If a split node, append left and right children and depth to `stack`
    # so we can loop through them
    if is_split_node:
        stack.append((children_left[node_id], depth + 1))
        stack.append((children_right[node_id], depth + 1))
    else:
        is_leaves[node_id] = True

print(
    "The binary tree structure has {n} nodes and has "
    "the following tree structure:\n".format(n=n_nodes)
)
for i in range(n_nodes):
    if is_leaves[i]:
        print(
            "{space}node={node} is a leaf node with value={value}.".format(
                space=node_depth[i] * "\t", node=i, value=values[i]
            )
        )
    else:
        print(
            "{space}node={node} is a split node with value={value}: "
            "go to node {left} if X[:, {feature}] <= {threshold} "
            "else to node {right}.".format(
                space=node_depth[i] * "\t",
                node=i,
                left=children_left[i],
                feature=feature[i],
                threshold=threshold[i],
                right=children_right[i],
                value=values[i],
            )
        )


sub plan:
 - we fit and save tree
 - access node and prob, align with classes
 - make an interactive python program 

note:
 - this indeed gives a dichotomous key, but sub tree may not always be explainable
 - this is easy and fast, and if training data is complete with no exception, you get a deterministic procedure **not just a probabilistic model**

In [ ]:
m

In [ ]:
plot_tree(m)

We build a sequence of entities

In [ ]:
import re
property_words = set(sum(list(map(lambda s: re.split('>', s),df2.columns.tolist())), []))
species_words = set(sum(list(map(lambda s: re.split('\s', s),df2.iloc[:,0].values)), []))
species_words ,property_words

all_words = species_words.union(property_words)


We can also account for mistyping or synonyms. In case for so many terms, one may have misspelled. 
Make a bench mark by sorting Levishtin edit distance to account for misspelled word.

let's try to see if there are 'usages'

In [ ]:
# !pip install editdistance

In [ ]:
import editdistance

query = 'usages'

distances = list(map(lambda word: editdistance.eval(query, word), list(all_words)))

rank = np.argsort(distances)
for i in range(5):
    print(list(all_words)[rank[i]], distances[rank[i]])


can also do a spell check with this 

In [ ]:
# !pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["usages", "usage"]

model = SentenceTransformer('sentence-transformers/LaBSE')
embeddings = model.encode(sentences)
print(embeddings)
